In [14]:
import os

json_file_path = "/Plant_Disease_Dataset_Unified"
print(os.path.exists(json_file_path))  # This should return True if the file exists


False


In [6]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:00:01m

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
print(os.listdir())

['.git', '.gitignore', 'Plant_Disease_Dataset_Unified', 'plant-image-identification-with-SAM.ipynb', '.ipynb_checkpoints', 'segment-anything', 'venv', 'requirements.txt']


In [9]:
import os
import json
import cv2
import numpy as np
from PIL import Image

def json_to_mask(json_file, output_dir):
    """
    Convert LabelMe JSON annotations to binary masks.
    :param json_file: Path to the JSON annotation file.
    :param output_dir: Directory to save the generated mask.
    """
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Get image dimensions
    img_width = data['imageWidth']
    img_height = data['imageHeight']
    mask = np.zeros((img_height, img_width), dtype=np.uint8)

    # Iterate over all annotated shapes (polygons)
    for shape in data['shapes']:
        points = np.array(shape['points'], dtype=np.int32)
        label = shape['label'].lower()  # Ensure case-insensitive comparison

        # Assign pixel values based on the label
        if label == "diseased":  # Replace with your actual label name
            cv2.fillPoly(mask, [points], 255)
        else:
            cv2.fillPoly(mask, [points], 0)  # Optional for multiple classes

    # Save the mask as a PNG file
    mask_filename = os.path.basename(json_file).replace('.json', '.png')
    mask_path = os.path.join(output_dir, mask_filename)
    mask_image = Image.fromarray(mask)
    mask_image.save(mask_path)

# Example usage
json_to_mask(
    json_file="/Plant_Disease_Dataset_Unified/train/images/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417_90deg.json",
    output_dir="/Plant_Disease_Dataset_Unified/train/images/Apple___Apple_scab/masks"
)

FileNotFoundError: [Errno 2] No such file or directory: '/Plant_Disease_Dataset_Unified/train/images/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417_90deg.json'

In [1]:
import torch
import os

print(os.listdir("./segment-anything"))
from segment_anything import sam_model_registry

# Load SAM model

sam_checkpoint = "segment-anything/sam_vit_h_4b8939.pth"  # Path to SAM weights model_type = "vit_h"  # Use "vit_b" for smaller models
device = "cuda" if torch.cuda.is_available() else "cpu"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)


['.git', '.flake8', '.gitignore', 'CODE_OF_CONDUCT.md', 'CONTRIBUTING.md', 'LICENSE', 'README.md', 'assets', 'demo', 'linter.sh', 'notebooks', 'scripts', 'segment_anything', 'setup.cfg', 'setup.py', 'sam_vit_h_4b8939.pth', 'segment_anything.egg-info']


NameError: name 'model_type' is not defined

In [5]:
import os
import cv2
import torch
from torch.utils.data import Dataset
from torchvision import transforms

class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.mask_paths = []

        # Load all image and mask paths
        for class_name in os.listdir(root_dir):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                mask_path = img_path.replace("images", "masks")  # Assuming masks are stored separately
                self.image_paths.append(img_path)
                self.mask_paths.append(mask_path)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]
    
        # Load image and mask
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    
        # Apply transformations
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']  # Already a tensor due to ToTensorV2()
            mask = augmented['mask']    # Already a tensor due to ToTensorV2()
    
        # Add channel dimension to mask
        mask = mask.unsqueeze(0)  # Shape: (1, H, W)
    
        return image, mask

ModuleNotFoundError: No module named 'cv2'

In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

transform = A.Compose([
    A.Resize(256, 256),  # Resize to 256x256
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # ImageNet normalization
    ToTensorV2(),
])

ModuleNotFoundError: No module named 'albumentations'

In [13]:
from torch.utils.data import DataLoader

# Define dataset paths
train_dataset = PlantDiseaseDataset(root_dir="Plant_Disease_Dataset_Unified/train", transform=transform)
valid_dataset = PlantDiseaseDataset(root_dir="Plant_Disease_Dataset_Unified/valid", transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

In [17]:
dataset = PlantDiseaseDataset(root_dir="Plant_Disease_Dataset_Unified/train", transform=transform)
image, mask = dataset[0]  # Get the first sample
print("Image shape:", image.shape)  # Should be (3, H, W)
print("Mask shape:", mask.shape)   # Should be (1, H, W)

Image shape: torch.Size([3, 256, 256])
Mask shape: torch.Size([1, 256, 256])


In [15]:
for images, masks in train_loader:
    print("Images shape:", images.shape)
    print("Masks shape:", masks.shape)
    break  # Stop after one batch

Images shape: torch.Size([8, 3, 256, 256])
Masks shape: torch.Size([8, 1, 256, 256])


In [20]:
import cv2
import numpy as np
from PIL import Image

def generate_mask(image_path, output_path):
    # Load an image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Could not load image at path: {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize SAM predictor
    predictor.set_image(image)

    # Define a point prompt
    input_point = np.array([[100, 100]])  # Point on diseased region
    input_label = np.array([1])  # 1 for foreground

    # Generate mask
    masks, _, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,  # Enable multi-mask output
    )

    # Save the mask
    mask_image = Image.fromarray((masks[0] * 255).astype(np.uint8))
    mask_image.save(output_path)

# Example usage
try:
    generate_mask("path/to/image.jpg", "path/to/output/mask.png")
except FileNotFoundError as e:
    print(e)

Could not load image at path: path/to/image.jpg


[ WARN:0@871.622] global loadsave.cpp:268 findDecoder imread_('path/to/image.jpg'): can't open/read file: check file path/integrity
